# Steal The Files ? Documentation (EN)
<span style="color:#4CAF50;font-weight:600">Client</span> ? <span style="color:#03A9F4;font-weight:600">Server</span> ? <span style="color:#FF9800;font-weight:600">Core</span> ? JavaFX 2D platformer

**Contents**
1. Overview & Architecture
2. Setup & Launch
3. Screens & Navigation
4. Controls, Camera & HUD
5. Gameplay Objects & Mechanics
6. Level Design
7. Audio & Settings
8. Networking & Data
9. Debugging & To-dos

## 1. Overview & Architecture
- Goal: grab the USB stick ? unlock & collect folder files ? survive hazards ? reach the finish portal.
- Modules: **core** (entity base, events, packets/serialization), **client** (JavaFX screens, levels, audio, input), **server** (TCP lobby, event router, DB hooks).
- Startup (Game.start): load config, apply sound settings, register event listeners, create ScreenManager + main/settings screens, set current level from `config.json`.
- Game loop: dedicated daemon thread (~200 Hz) in ScreenManager computes `delta` + FPS and calls `currentScreen.update(delta)` on the JavaFX thread.
- Runtime pipeline:
```text
Input -> InputManager -> ScreenManager.update(delta) -> GameScreen.update(delta)
  -> Physics (dx/dy, gravity)
  -> Collision (platforms + blocks)
  -> Camera (dead zone + smoothing)
  -> HUD (hearts, quest, debug)
  -> Render + audio (SoundManager)
```
- Level selection via config (Tutorial/Second/Boss):
```java
switch (config.getString("currentLevel")) {
    case "Tutorial" -> setCurrentLevel(new TutorialLevel());
    case "Second"   -> setCurrentLevel(new SecondLevel());
    case "Boss"     -> setCurrentLevel(new BossLevel());
}
```

## 2. Setup & Launch
- Requirements: JDK 19 (Corretto), JavaFX 17, Maven; assets live under `/assets`.
- Build all modules:
```bash
mvn clean install
```
- Run (IDE-friendly):
  - Server: `de.cyzetlc.hsbi.Server` (opens TCP 25570, reads `server/config.json`).
  - Client: `de.cyzetlc.hsbi.game.Game` (loads config, connects to server, shows Loading ? MainMenu).
- Config (`config.json`, secrets masked):
```json
{
  "currentLevel": "Tutorial",
  "soundVolume": 0.6,
  "soundMuted": false,
  "messages": { "game.prefix": "HSBI-GAME" },
  "mysql": {
    "hostname": "db-host",
    "database": "hsbi",
    "username": "user",
    "password": "<secret>",
    "port": 3306,
    "poolSize": 3
  },
  "tutorialFinished": false
}
```

## 3. Screens & Navigation
- Flow: `Loading` ? `MainMenu` ? {`Settings` | `Multiplayer` | `Start Game`} ? `GameScreen` ? `LevelFinished` ? {next level | main menu}.
- **LoadingScreen**: animated background + progress bar; warms all `Material` textures, then switches to menu.
- **MainMenuScreen**: parallax background, buttons (Start/Multiplayer/Settings/Exit), achievements panel (dummy progress).
- **MultiplayerScreen**: mock server list (slots + MOTD) with back navigation.
- **SettingsScreen**: sound panel (0?1 volume slider with `custom-slider` style, mute toggle) persisting to `config.json`.
- **GameScreen**: core gameplay, camera + HUD, Back/Pause buttons, quest text, file counter, health bar; pause overlay currently visual only.
- **LevelFinishedScreen**: stats overlay (time, score, kills, lives lost) + buttons for next level or main menu.

## 4. Controls, Camera & HUD
- Movement: `A/D` move, `SPACE` jump (grounded), `E` interact (gas barrier), `W/S` in NoClip.
- Debug/states: `F1` tooltips, `F2` debug bar (FPS, camera, HP, states), `F3` NoClip + GodMode.
- Camera: dead zone (`marginX=400`, `marginY=150`), smoothing `cameraSmooth=0.1`, offset applied to all sprites.
- HUD: quest text, files progress (`Files: collected/total`), flipper hint, health bar (hearts from `player.getMaxHealth()`=3), optional debug bar.
- Input debouncing via `InputManager.pollJustPressed()`, e.g.
```java
if (screenManager.getInputManager().pollJustPressed(KeyCode.F2)) {
    showDebugBar = !showDebugBar;
}
```

## 5. Gameplay Objects & Mechanics
- **Player & states**: `EntityPlayer` extends `Player` (base HP 3, name tag, direction-based sprite flip, `canCollectFiles` flag); GodMode/NoClip ignore damage and collision.
- **Items & collectibles**: USB stick unlocks file collection; `FolderBlock` only counts after unlock; `FlipperItem` sets key flag (HUD hint) to disable gas barrier; `ServerBlock` is animated decoration.
- **Perks**: `JumpBoostBlock` and `SpeedBoostBlock` add 25% jump power/move speed for 10s (via `PauseTransition`), then reset and hide.
- **Hazards & enemies**: `LavaBlock` ticks 0.25 HP (no damage in GodMode); `GasBarrierBlock` kills/blocks until E + flipper, fades out; `RobotEnemyBlock` patrols and fires `LaserBlock` (4s lifetime, 1 dmg, removed even in GodMode); Boss level chains multiple robots.
- **Platforms**: `Platform` (static, tiled), `FloatingPlatformBlock` (ping-pong between start/end, animated, carries player via `deltaX`), `EntityMovingPlatform` (sinus hover + ping-pong, supports landing helper).

## 6. Level Design
- **TutorialLevel**: staggered platforms, lava gaps filled, early USB, jump/speed boosts, server block, floating platform linking islands, finish on the right; `onFinish()` sets `tutorialFinished=true`.
- **SecondLevel**: denser layout, 8 folders, jump-boost route, flipper near start, boss robot on final platform, finish at far right; gas barrier prepared (currently commented), one-time spawn sound.
- **BossLevel**: wide start platform with four RobotEnemyBlocks as a mini-boss lane, finish beyond them.
- Start level is read from `config.json` (`currentLevel`); next-level chain: Tutorial ? Second ? Boss ? end.

## 7. Audio & Settings
- `SoundManager`: media caching, global volume (`setVolume` 0?1), mute flag, background music (`playBackground(Music.MENU, true)`), ducking for SFX (`playWithDuck`).
- SettingsScreen mirrors sound state: slider updates volume live, mute disables slider and toggles text (`Sound AN`/`Mute`), persists directly to config.
- SFX: clicks on pickup/interaction, buff cues, lasers/robots; menu music loops in the background.
- Asset prep: LoadingScreen warms all block textures to avoid first-frame stutter.

## 8. Networking & Data
- Client opens TCP to `localhost:25570`, sends login (`ClientLoginPacket`) and heartbeat (`ClientDataPacket` every 50ms).
- Events: `ReceivePacketEvent` + listeners (Packet/UserMessage/Player) dispatch incoming packets on the client.
- Score submission: `sendFinalScore(long)` issues `ClientSubmitScorePacket`; server can broadcast highscores via `ServerSendHighscoresPacket` (hook present).
- Server: MultiClientHandler per socket, periodic status message, deserializes via `SerializationUtils`, uses `JsonConfig` + `QueryHandler` (MySQL creds in config) for DB wiring.
- Packets are serialized (max ~0.5 MB) and cancellable through the event system (`EventCancelable`).

## 9. Debugging & To-dos
- Live checks: F1/F2/F3 toggles, pause overlay (logic not paused yet), health bar reacts to damage/heal, file counter after USB unlock.
- Manual test ideas:
  - Tutorial: pick up USB ? collect folder ? finish triggers LevelFinishedScreen.
  - Buffs: jump/speed boosts last 10s, values revert correctly.
  - Enemies: RobotEnemyBlock can be stomped (from above) or deals damage; lasers vanish after 4s.
  - Flipper + GasBarrier (if placed) disables the barrier with E and fades it out.
- Open items: real pause (stop update loop), leverage `Level.update()` for moving blocks, per-level camera clamps, map LevelFinished stats to real data, finalize server integration for multiplayer/highscores.